# Understanding Bayes Nets

We have:

- Nodes: Variables (features like Age, Stress_Level and Mental_Health_Condition)
- Edges: Dependencies between variables (Stress_Level -> Mental_Health_Condition)

Bayes Nets encode the joint probability distribution of variables, allowing inference given evidence.

# Setup

In [2]:
from inspect import getsource
from IPython.display import display
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import itertools
import math
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot

In [3]:
random.seed(42)

In [4]:
mental_data = pd.read_csv("../data/cleaned_breast_cancer.csv")

In [5]:
mental_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    int64  
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [7]:
train_data, test_data = train_test_split(
    mental_data,
    test_size=0.4,
    random_state=42,
    stratify=mental_data["diagnosis"],
)

In [8]:
categorical_columns = mental_data.columns
for col in categorical_columns:
    train_data[col] = train_data[col].astype("category")
    test_data[col] = test_data[col].astype("category")

# Helper Code

In [9]:
def extend(s, var, val):
    """Copy dict s and extend it by setting var to val; return copy."""
    return {**s, var: val}

In [10]:
def consistent_with(event, evidence):
    """Is event consistent with the given evidence?"""
    return all(evidence.get(k, v) == v
               for k, v in event.items())

In [11]:
def probability(p):
    """Return true with probability p."""
    return p > random.uniform(0.0, 1.0)

In [12]:
class ProbDist:
    """A discrete probability distribution. You name the random variable
    in the constructor, then assign and query probability of values.
    >>> P = ProbDist('Flip'); P['H'], P['T'] = 0.25, 0.75; P['H']
    0.25
    >>> P = ProbDist('X', {'lo': 125, 'med': 375, 'hi': 500})
    >>> P['lo'], P['med'], P['hi']
    (0.125, 0.375, 0.5)
    """

    def __init__(self, varname='?', freqs=None):
        """If freqs is given, it is a dictionary of values - frequency pairs,
        then ProbDist is normalized."""
        self.prob = {}
        self.varname = varname
        self.values = []
        if freqs:
            for (v, p) in freqs.items():
                self[v] = p
            self.normalize()

    def __getitem__(self, val):
        """Given a value, return P(value)."""
        try:
            return self.prob[val]
        except KeyError:
            return 0

    def __setitem__(self, val, p):
        """Set P(val) = p."""
        if val not in self.values:
            self.values.append(val)
        self.prob[val] = p

    def normalize(self):
        """Make sure the probabilities of all values sum to 1.
        Returns the normalized distribution.
        Raises a ZeroDivisionError if the sum of the values is 0."""
        total = sum(self.prob.values())
        if not np.isclose(total, 1.0):
            for val in self.prob:
                self.prob[val] /= total
        return self

    def show_approx(self, numfmt='{:.3g}'):
        """Show the probabilities rounded and sorted by key, for the
        sake of portable doctests."""
        return ', '.join([('{}: ' + numfmt).format(v, p)
                          for (v, p) in sorted(self.prob.items())])

    def __repr__(self):
        return "P({})".format(self.varname)

In [13]:
class JointProbDist(ProbDist):
    """A discrete probability distribute over a set of variables.
    >>> P = JointProbDist(['X', 'Y']); P[1, 1] = 0.25
    >>> P[1, 1]
    0.25
    >>> P[dict(X=0, Y=1)] = 0.5
    >>> P[dict(X=0, Y=1)]
    0.5"""

    def __init__(self, variables):
        self.prob = {}
        self.variables = variables
        self.vals = defaultdict(list)

    def __getitem__(self, values):
        """Given a tuple or dict of values, return P(values)."""
        values = event_values(values, self.variables)
        return ProbDist.__getitem__(self, values)

    def __setitem__(self, values, p):
        """Set P(values) = p.  Values can be a tuple or a dict; it must
        have a value for each of the variables in the joint. Also keep track
        of the values we have seen so far for each variable."""
        values = event_values(values, self.variables)
        self.prob[values] = p
        for var, val in zip(self.variables, values):
            if val not in self.vals[var]:
                self.vals[var].append(val)

    def values(self, var):
        """Return the set of possible values for a variable."""
        return self.vals[var]

    def __repr__(self):
        return "P({})".format(self.variables)

In [14]:
class BayesNode:
    """A conditional probability distribution for a boolean variable,
    P(X | parents). Part of a BayesNet."""

    def __init__(self, X, parents, cpt):
        """X is a variable name, and parents a sequence of variable
        names or a space-separated string.  cpt, the conditional
        probability table, takes one of these forms:

        * A number, the unconditional probability P(X=true). You can
          use this form when there are no parents.

        * A dict {v: p, ...}, the conditional probability distribution
          P(X=true | parent=v) = p. When there's just one parent.

        * A dict {(v1, v2, ...): p, ...}, the distribution P(X=true |
          parent1=v1, parent2=v2, ...) = p. Each key must have as many
          values as there are parents. You can use this form always;
          the first two are just conveniences.

        In all cases the probability of X being false is left implicit,
        since it follows from P(X=true).

        >>> X = BayesNode('X', '', 0.2)
        >>> Y = BayesNode('Y', 'P', {T: 0.2, F: 0.7})
        >>> Z = BayesNode('Z', 'P Q',
        ...    {(T, T): 0.2, (T, F): 0.3, (F, T): 0.5, (F, F): 0.7})
        """
        if isinstance(parents, str):
            parents = parents.split()

        # We store the table always in the third form above.
        if isinstance(cpt, (float, int)):  # no parents, 0-tuple
            cpt = {(): cpt}
        elif isinstance(cpt, dict):
            # one parent, 1-tuple
            if cpt and isinstance(list(cpt.keys())[0], bool):
                cpt = {(v,): p for v, p in cpt.items()}

        assert isinstance(cpt, dict)
        for vs, p in cpt.items():
            assert isinstance(vs, tuple) and len(vs) == len(parents)
            assert all(isinstance(v, bool) for v in vs)
            assert 0 <= p <= 1

        self.variable = X
        self.parents = parents
        self.cpt = cpt
        self.children = []

    def p(self, value, event):
        """Return the conditional probability
        P(X=value | parents=parent_values), where parent_values
        are the values of parents in event. (event must assign each
        parent a value.)
        >>> bn = BayesNode('X', 'Burglary', {T: 0.2, F: 0.625})
        >>> bn.p(False, {'Burglary': False, 'Earthquake': True})
        0.375"""
        assert isinstance(value, bool)
        ptrue = self.cpt[event_values(event, self.parents)]
        return ptrue if value else 1 - ptrue

    def sample(self, event):
        """Sample from the distribution for this variable conditioned
        on event's values for parent_variables. That is, return True/False
        at random according with the conditional probability given the
        parents."""
        return probability(self.p(True, event))

    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))

In [15]:
def probability_sampling(probabilities):
    """Randomly sample an outcome from a probability distribution."""
    total = sum(probabilities.values())
    r = random.uniform(0, total)
    cumulative = 0
    for outcome, prob in probabilities.items():
        cumulative += prob
        if r <= cumulative:
            return outcome
    return None  # Should not reach here if probabilities are normalize

In [16]:
class MultiClassBayesNode:
    """A Bayesian node for multi-class variables."""
    def __init__(self, X, parents, cpt):
        """
        X: Name of the variable.
        parents: List of parent variable names.
        cpt: Conditional probability table, mapping tuples of parent values to
             dictionaries of target probabilities.
        """
        if isinstance(parents, str):
            parents = parents.split()
        self.variable = X
        self.parents = parents
        self.cpt = cpt
        self.children = []

    def p(self, value, event):
        """Return the conditional probability P(X=value | parents=parent_values)."""
        parent_values = tuple(event.get(p, None) for p in self.parents)
        probabilities = self.cpt.get(parent_values, {})
        return probabilities.get(value, 0)  # Default to 0 if value not found

    def sample(self, event):
        """Sample from the distribution for this variable given parent values."""
        parent_values = tuple(event.get(p, None) for p in self.parents)
        probabilities = self.cpt.get(parent_values, {})
        return probability_sampling(probabilities)

    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))

In [17]:
class BayesNet:
    """Bayesian network containing only boolean-variable or multi-class nodes."""
    
    def __init__(self, node_specs=None):
        """Nodes must be ordered with parents before children."""
        self.nodes = []
        self.variables = []
        node_specs = node_specs or []
        for node_spec in node_specs:
            self.add(node_spec)

    def add(self, node_spec):
        """Add a node to the net. Supports both pre-constructed nodes and node specs."""
        if isinstance(node_spec, (BayesNode, MultiClassBayesNode)):
            # If already a node, add it directly
            node = node_spec
        else:
            # Otherwise, initialize a new node
            node = BayesNode(*node_spec)

        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)

        self.nodes.append(node)
        self.variables.append(node.variable)

        # Register children for the parent nodes
        for parent in node.parents:
            self.variable_node(parent).children.append(node)

    def variable_node(self, var):
        """Return the node for the variable named var."""
        for n in self.nodes:
            if n.variable == var:
                return n
        raise Exception(f"No such variable: {var}")

    def variable_values(self, var):
        """Return the domain of var."""
        return [True, False]

    def __repr__(self):
        return f"BayesNet({self.nodes!r})"

In [18]:
def make_factor(var, e, bn):
    """Return the factor for var in bn's joint distribution given e.
    That is, bn's full joint distribution, projected to accord with e,
    is the pointwise product of these factors for bn's variables."""
    node = bn.variable_node(var)
    variables = [X for X in [var] + node.parents if X not in e]
    cpt = {event_values(e1, variables): node.p(e1[var], e1)
           for e1 in all_events(variables, bn, e)}
    return Factor(variables, cpt)

In [19]:
class Factor:
    """A factor in a joint distribution."""

    def __init__(self, variables, cpt):
        self.variables = variables
        self.cpt = cpt

    def pointwise_product(self, other, bn):
        """Multiply two factors, combining their variables."""
        variables = list(set(self.variables) | set(other.variables))
        cpt = {event_values(e, variables): self.p(e) * other.p(e) for e in all_events(variables, bn, {})}
        return Factor(variables, cpt)

    def sum_out(self, var, bn):
        """Make a factor eliminating var by summing over its values."""
        variables = [X for X in self.variables if X != var]
        cpt = {event_values(e, variables): sum(self.p(extend(e, var, val)) for val in bn.variable_values(var))
               for e in all_events(variables, bn, {})}
        return Factor(variables, cpt)

    def normalize(self):
        """Return my probabilities; must be down to one variable."""
        assert len(self.variables) == 1
        return ProbDist(self.variables[0], {k: v for ((k,), v) in self.cpt.items()})

    def p(self, e):
        """Look up my value tabulated for e."""
        return self.cpt[event_values(e, self.variables)]

In [20]:
def enumerate_all(variables, e, bn):
    """Return the sum of those entries in P(variables | e{others})
    consistent with e, where P is the joint distribution represented
    by bn, and e{others} means e restricted to bn's other variables
    (the ones other than variables). Parents must precede children in variables."""
    if not variables:
        return 1.0
    Y, rest = variables[0], variables[1:]
    Ynode = bn.variable_node(Y)
    if Y in e:
        return Ynode.p(e[Y], e) * enumerate_all(rest, e, bn)
    else:
        return sum(Ynode.p(y, e) * enumerate_all(rest, extend(e, Y, y), bn)
                   for y in bn.variable_values(Y))

In [21]:
def enumeration_ask(X, e, bn):
    """Return the conditional probability distribution of variable X
    given evidence e, from BayesNet bn. [Figure 14.9]
    >>> enumeration_ask('Burglary', dict(JohnCalls=T, MaryCalls=T), burglary
    ...  ).show_approx()
    'False: 0.716, True: 0.284'"""
    assert X not in e, "Query variable must be distinct from evidence"
    Q = ProbDist(X)
    for xi in bn.variable_values(X):
        Q[xi] = enumerate_all(bn.variables, extend(e, X, xi), bn)
    return Q.normalize()

In [22]:
def elimination_ask(X, e, bn):
    """Compute bn's P(X|e) by variable elimination. [Figure 14.11]
    >>> elimination_ask('Burglary', dict(JohnCalls=T, MaryCalls=T), burglary
    ...  ).show_approx()
    'False: 0.716, True: 0.284'"""
    assert X not in e, "Query variable must be distinct from evidence"
    factors = []
    for var in reversed(bn.variables):
        factors.append(make_factor(var, e, bn))
        if is_hidden(var, X, e):
            factors = sum_out(var, factors, bn)
    return pointwise_product(factors, bn).normalize()

In [23]:
def prior_sample(bn):
    """Randomly sample from bn's full joint distribution. The result
    is a {variable: value} dict. [Figure 14.13]"""
    event = {}
    for node in bn.nodes:
        event[node.variable] = node.sample(event)
    return event

In [24]:
def rejection_sampling(X, e, bn, N=10000):
    """Estimate the probability distribution of variable X given
    evidence e in BayesNet bn, using N samples.  [Figure 14.14]
    Raises a ZeroDivisionError if all the N samples are rejected,
    i.e., inconsistent with e.
    >>> random.seed(47)
    >>> rejection_sampling('Burglary', dict(JohnCalls=T, MaryCalls=T),
    ...   burglary, 10000).show_approx()
    'False: 0.7, True: 0.3'
    """
    counts = {x: 0 for x in bn.variable_values(X)}  # bold N in [Figure 14.14]
    for j in range(N):
        sample = prior_sample(bn)  # boldface x in [Figure 14.14]
        if consistent_with(sample, e):
            counts[sample[X]] += 1
    return ProbDist(X, counts)

In [25]:
def likelihood_weighting(X, e, bn, N=10000):
    """Estimate the probability distribution of variable X given
    evidence e in BayesNet bn.  [Figure 14.15]
    >>> random.seed(1017)
    >>> likelihood_weighting('Burglary', dict(JohnCalls=T, MaryCalls=T),
    ...   burglary, 10000).show_approx()
    'False: 0.702, True: 0.298'
    """
    W = {x: 0 for x in bn.variable_values(X)}
    for j in range(N):
        sample, weight = weighted_sample(bn, e)  # boldface x, w in [Figure 14.15]
        W[sample[X]] += weight
    return ProbDist(X, W)

In [26]:
def gibbs_ask(X, e, bn, N=1000):
    """[Figure 14.16]"""
    assert X not in e, "Query variable must be distinct from evidence"
    counts = {x: 0 for x in bn.variable_values(X)}  # bold N in [Figure 14.16]
    Z = [var for var in bn.variables if var not in e]
    state = dict(e)  # boldface x in [Figure 14.16]
    for Zi in Z:
        state[Zi] = random.choice(bn.variable_values(Zi))
    for j in range(N):
        for Zi in Z:
            state[Zi] = markov_blanket_sample(Zi, state, bn)
            counts[state[X]] += 1
    return ProbDist(X, counts)

In [27]:
def pointwise_product(self, other, bn):
        """Multiply two factors, combining their variables."""
        variables = list(set(self.variables) | set(other.variables))
        cpt = {event_values(e, variables): self.p(e) * other.p(e)
               for e in all_events(variables, bn, {})}
        return Factor(variables, cpt)

In [28]:
def sum_out(self, var, bn):
        """Make a factor eliminating var by summing over its values."""
        variables = [X for X in self.variables if X != var]
        cpt = {event_values(e, variables): sum(self.p(extend(e, var, val))
                                               for val in bn.variable_values(var))
               for e in all_events(variables, bn, {})}
        return Factor(variables, cpt)

In [29]:
def event_values(event, variables):
    """Return a tuple of the values of variables in event.
    >>> event_values ({'A': 10, 'B': 9, 'C': 8}, ['C', 'A'])
    (8, 10)
    >>> event_values ((1, 2), ['C', 'A'])
    (1, 2)
    """
    if isinstance(event, tuple) and len(event) == len(variables):
        return event
    else:
        return tuple([event[var] for var in variables])

# Design the Network Structure

Find dependencies.

# Estimate Conditional Probabilities

If a node is a root node, then estimate probability directly from the data. Estimate conditional probabilities based on parent for non-root nodes.

In [32]:
def compute_cpt(data, target, parents, alpha=1):
    """
    Compute CPT with Laplace smoothing.
    
    Args:
        data: pandas DataFrame (training data)
        target: str, target variable
        parents: list of parent variable names
        alpha: smoothing parameter (default=1)
    
    Returns:
        cpt: dict { parent_values_tuple: { target_value: probability } }
    """
    target_values = data[target].cat.categories

    if not parents:
        # Marginal distribution of target
        counts = defaultdict(lambda: alpha)
        for val in data[target]:
            counts[val] += 1
        total = sum(counts.values())
        cpt = {(): {tv: counts[tv]/total for tv in counts}}
        return cpt

    # Determine possible parent combinations
    from itertools import product
    parent_values_list = [data[p].cat.categories for p in parents]
    parent_combinations = list(product(*parent_values_list)) if parents else [()]

    # Initialize counts with alpha
    counts = {pc: defaultdict(lambda: alpha) for pc in parent_combinations}

    # Count occurrences
    for _, row in data.iterrows():
        pv = tuple(row[p] for p in parents) if parents else ()
        tv = row[target]
        counts[pv][tv] += 1

    # Compute probabilities
    cpt = {}
    for pc in parent_combinations:
        total = sum(counts[pc].values())
        cpt[pc] = {tv: (counts[pc][tv] / total) for tv in counts[pc]}
        
    return cpt

In [79]:
import time

In [81]:
def compute_all_cpts(train_data):
    """
    Compute all CPTs and record the time taken.
    """
    start_time = time.time()  # Start timing
    
    # Compute CPTs
    cpt_diagnosis = compute_cpt(train_data, 'diagnosis', [])
    cpt_concave_points = compute_cpt(train_data, 'concave points_mean', ['diagnosis'])
    cpt_perimeter = compute_cpt(train_data, 'perimeter_mean', ['diagnosis'])
    cpt_radius = compute_cpt(train_data, 'radius_mean', ['diagnosis'])
    cpt_concavity = compute_cpt(train_data, 'concavity_mean', ['diagnosis', 'concave points_mean'])
    cpt_texture = compute_cpt(train_data, 'texture_mean', ['diagnosis'])

    cpt_area = compute_cpt(train_data, 'area_mean', ['perimeter_mean'])
    cpt_compactness = compute_cpt(train_data, 'compactness_mean', ['concavity_mean'])
    cpt_smoothness = compute_cpt(train_data, 'smoothness_mean', ['concavity_mean'])
    cpt_symmetry = compute_cpt(train_data, 'symmetry_mean', ['compactness_mean'])
    cpt_fractal = compute_cpt(train_data, 'fractal_dimension_mean', ['symmetry_mean'])
    
    end_time = time.time()  # End timing
    training_time = end_time - start_time
    print(f"Training Time (CPT Computation): {training_time:.4f} seconds")

    return {
        "cpt_diagnosis": cpt_diagnosis,
        "cpt_concave_points": cpt_concave_points,
        "cpt_perimeter": cpt_perimeter,
        "cpt_radius": cpt_radius,
        "cpt_concavity": cpt_concavity,
        "cpt_texture": cpt_texture,
        "cpt_area": cpt_area,
        "cpt_compactness": cpt_compactness,
        "cpt_smoothness": cpt_smoothness,
        "cpt_symmetry": cpt_symmetry,
        "cpt_fractal": cpt_fractal
    }, training_time

In [82]:
    # ('diagnosis', 'concave points_mean'),
    # ('diagnosis', 'perimeter_mean'),
    # ('diagnosis', 'radius_mean'),
    # ('diagnosis', 'concavity_mean'),
    # ('diagnosis', 'texture_mean'),
    
    # ('concave points_mean', 'concavity_mean'),
    # ('concavity_mean', 'compactness_mean'),
    # ('concavity_mean', 'smoothness_mean'),
    
    # ('radius_mean', 'perimeter_mean'),
    # ('perimeter_mean', 'area_mean'),
    
    # ('compactness_mean', 'symmetry_mean'),
    # ('symmetry_mean', 'fractal_dimension_mean')

In [83]:
# cpt_diagnosis = compute_cpt(train_data, 'diagnosis', [])
# cpt_concave_points = compute_cpt(train_data, 'concave points_mean', ['diagnosis'])
# cpt_perimeter = compute_cpt(train_data, 'perimeter_mean', ['diagnosis'])
# cpt_radius = compute_cpt(train_data, 'radius_mean', ['diagnosis'])
# cpt_concavity = compute_cpt(train_data, 'concavity_mean', ['diagnosis', 'concave points_mean'])
# cpt_texture = compute_cpt(train_data, 'texture_mean', ['diagnosis'])

# cpt_area = compute_cpt(train_data, 'area_mean', ['perimeter_mean'])
# cpt_compactness = compute_cpt(train_data, 'compactness_mean', ['concavity_mean'])
# cpt_smoothness = compute_cpt(train_data, 'smoothness_mean', ['concavity_mean'])
# cpt_symmetry = compute_cpt(train_data, 'symmetry_mean', ['compactness_mean'])
# cpt_fractal = compute_cpt(train_data, 'fractal_dimension_mean', ['symmetry_mean'])

# Implement Inference

Use the chain rule of probability to predict target given evidence.

In [84]:
cpts, training_time = compute_all_cpts(train_data)

Training Time (CPT Computation): 0.1161 seconds


In [85]:
diagnosis_node = MultiClassBayesNode("diagnosis", [], cpts['cpt_diagnosis'])

concave_points_node = MultiClassBayesNode(
    "concave points_mean", ["diagnosis"], cpts['cpt_concave_points']
)
perimeter_node = MultiClassBayesNode("perimeter_mean", ["diagnosis"], cpts['cpt_perimeter'])
radius_node = MultiClassBayesNode("radius_mean", ["diagnosis"], cpts['cpt_radius'])
concavity_node = MultiClassBayesNode(
    "concavity_mean", ["diagnosis", "concave points_mean"], cpts['cpt_concavity'])

texture_node = MultiClassBayesNode("texture_mean", ["diagnosis"], cpts['cpt_texture'])

area_node = MultiClassBayesNode("area_mean", ["perimeter_mean"], cpts['cpt_area'])
compactness_node = MultiClassBayesNode("compactness_mean", ["concavity_mean"], cpts['cpt_compactness'])
smoothness_node = MultiClassBayesNode("smoothness_mean", ["concavity_mean"], cpts['cpt_smoothness'])
symmetry_node = MultiClassBayesNode("symmetry_mean", ["compactness_mean"], cpts['cpt_symmetry'])
fractal_node = MultiClassBayesNode("fractal_dimension_mean", ["symmetry_mean"], cpts['cpt_fractal'])

In [86]:
diagnosis_bn = BayesNet([
    diagnosis_node,
    concave_points_node,
    perimeter_node,
    radius_node,
    concavity_node,
    texture_node,
    area_node,
    compactness_node,
    smoothness_node,
    symmetry_node,
    fractal_node
])

In [69]:
print(diagnosis_bn)

BayesNet([('diagnosis', ''), ('concave points_mean', 'diagnosis'), ('perimeter_mean', 'diagnosis'), ('radius_mean', 'diagnosis'), ('concavity_mean', 'diagnosis concave points_mean'), ('texture_mean', 'diagnosis'), ('area_mean', 'perimeter_mean'), ('compactness_mean', 'concavity_mean'), ('smoothness_mean', 'concavity_mean'), ('symmetry_mean', 'compactness_mean'), ('fractal_dimension_mean', 'symmetry_mean')])


# Evaluate the Model

In [70]:
def predict_bayes_net(bn, evidence, query_var):
    """
    Predict the most likely value of a query variable given evidence using the Bayesian Network.
    
    Args:
        bn: Bayesian network.
        evidence: Dictionary of evidence variables and their values.
        query_var: Variable to predict.
    
    Returns:
        The most likely value of the query variable.
    """
    result = enumeration_ask(query_var, evidence, bn)
    return max(result.prob, key=lambda k: result.prob[k])

In [75]:
def evaluate_bayes_net(bn, test_data, query_var):
    """
    Evaluate the Bayesian Network on a test dataset.
    
    Args:
        bn: Bayesian Network (BayesNet instance).
        test_data: Test dataset (pandas DataFrame).
        query_var: The target variable to predict.
    
    Returns:
        Accuracy of the predictions as a float.
    """
    correct = 0  # Count of correct predictions

    for _, row in test_data.iterrows():
        # Build evidence dictionary from test row
        evidence = {
            "concave points_mean": row["concave points_mean"],
            "perimeter_mean": row["perimeter_mean"],
            "radius_mean": row["radius_mean"],
            "concavity_mean": row["concavity_mean"],
            "texture_mean": row["texture_mean"],
            "area_mean": row["area_mean"],
            "compactness_mean": row["compactness_mean"],
            "smoothness_mean": row["smoothness_mean"],
            "symmetry_mean": row["symmetry_mean"],
            "fractal_dimension_mean": row["fractal_dimension_mean"],
        }

        # Predict the target variable
        prediction = predict_bayes_net(bn, evidence, query_var)
        
        
        # Check if the prediction matches the actual value
        if prediction == row[query_var]:
            correct += 1

    # Calculate accuracy
    accuracy = correct / len(test_data)
    return accuracy

In [76]:
# Evaluate the Bayesian Network on the test dataset
query_var = "diagnosis"
accuracy = evaluate_bayes_net(diagnosis_bn, test_data, query_var)

print(f"Accuracy of Bayesian Network: {accuracy:.2%}")

Accuracy of Bayesian Network: 90.35%


In [87]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def evaluate_bayes_net(bn, test_data, query_var):
    """
    Evaluate the Bayesian Network on a test dataset and compute various metrics.

    Args:
        bn: Bayesian Network (BayesNet instance).
        test_data: Test dataset (pandas DataFrame).
        query_var: The target variable to predict.

    Returns:
        metrics: A dictionary containing accuracy, and prints out confusion matrix
                 and classification report (precision, recall, f1).
    """

    # Start Timing Predictions
    start_time = time.time()

    y_true = []
    y_pred = []

    for _, row in test_data.iterrows():
        # Build evidence dictionary from test row
        # Note: Adjust the evidence set according to what you want to condition on.
        evidence = {
            "concave points_mean": row["concave points_mean"],
            "perimeter_mean": row["perimeter_mean"],
            "radius_mean": row["radius_mean"],
            "concavity_mean": row["concavity_mean"],
            "texture_mean": row["texture_mean"],
            "area_mean": row["area_mean"],
            "compactness_mean": row["compactness_mean"],
            "smoothness_mean": row["smoothness_mean"],
            "symmetry_mean": row["symmetry_mean"],
            "fractal_dimension_mean": row["fractal_dimension_mean"],
        }

        # Predict the target variable
        prediction = predict_bayes_net(bn, evidence, query_var)

        y_true.append(row[query_var])
        y_pred.append(prediction)

    end_time = time.time()
    prediction_time = end_time - start_time    

    # Calculate accuracy
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred, zero_division=0)

    print("Confusion Matrix:\n", cm)
    print("\nClassification Report:\n", report)
    print(f"Prediction Time: {prediction_time:.4f} seconds")

    metrics = {
        "accuracy": acc,
        "prediction_time": prediction_time
    }

    return metrics

In [88]:
metrics = evaluate_bayes_net(diagnosis_bn, test_data, "diagnosis")
print("Accuracy:", metrics["accuracy"])

Confusion Matrix:
 [[134   9]
 [ 13  72]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.91      0.94      0.92       143
         1.0       0.89      0.85      0.87        85

    accuracy                           0.90       228
   macro avg       0.90      0.89      0.90       228
weighted avg       0.90      0.90      0.90       228

Prediction Time: 0.0253 seconds
Accuracy: 0.9035087719298246


In [89]:
cpts

{'cpt_diagnosis': {(): {1: 0.37317784256559766, 0: 0.6268221574344023}},
 'cpt_concave_points': {(0,): {0.0: 0.5253456221198156,
   1.0: 0.42857142857142855,
   2.0: 0.04608294930875576},
  (1,): {2.0: 0.7923076923076923,
   1.0: 0.18461538461538463,
   0.0: 0.023076923076923078}},
 'cpt_perimeter': {(0,): {0.0: 0.5253456221198156,
   1.0: 0.41013824884792627,
   2.0: 0.06451612903225806},
  (1,): {2.0: 0.8, 1.0: 0.15384615384615385, 0.0: 0.046153846153846156}},
 'cpt_radius': {(0,): {0.0: 0.5023041474654378,
   1.0: 0.42857142857142855,
   2.0: 0.06912442396313365},
  (1,): {2.0: 0.8, 1.0: 0.14615384615384616, 0.0: 0.05384615384615385}},
 'cpt_concavity': {(0, 0.0): {0.0: 0.7844827586206896,
   1.0: 0.19827586206896552,
   2.0: 0.017241379310344827},
  (0, 1.0): {1.0: 0.6631578947368421,
   0.0: 0.24210526315789474,
   2.0: 0.09473684210526316},
  (0, 2.0): {2.0: 0.6363636363636364, 1.0: 0.36363636363636365},
  (1, 0.0): {0.0: 1.0},
  (1, 1.0): {0.0: 0.19230769230769232,
   1.0: 0.576

In [90]:
def cpts_to_json(cpts):
    serializable_cpts = {}
    for var, cpt in cpts.items():
        serializable_cpts[var] = {
            str(parent_comb): {str(k): v for k, v in target_probs.items()}
            for parent_comb, target_probs in cpt.items()
        }
    return serializable_cpts

In [ ]:
# Save the CPTs to a JSON file
cpt_json = cpts_to_json(cpts)
with open("cancer_cpts.json", "w") as f:
    json.dump(cpt_json, f, indent=4)
print("CPTs saved successfully to covid_cpts.json!")

In [ ]:
cpt_json